In [1]:
from pathlib import Path
import pandas as pd

BASE_DIR = Path.cwd()          # current working directory
DATA_DIR = BASE_DIR / "data"
MIN_ORDERS = 5

In [2]:
# 1. Load tables
customers = pd.read_csv(DATA_DIR / "olist_customers_dataset.csv")
orders    = pd.read_csv(DATA_DIR / "olist_orders_dataset.csv")
items     = pd.read_csv(DATA_DIR / "olist_order_items_dataset.csv")
reviews   = pd.read_csv(DATA_DIR / "olist_order_reviews_dataset.csv")
payments  = pd.read_csv(DATA_DIR / "olist_order_payments_dataset.csv")

# 2. Filter completed orders
orders_ok = orders[
    (orders.order_status == "delivered") &
    (orders.order_delivered_customer_date.notna())
]

# 3. Attach customer identity
orders_ok = orders_ok.merge(
    customers[["customer_id", "customer_unique_id"]],
    on="customer_id",
    how="left"
)

# 4. Aggregate items → order level
items_agg = (
    items
    .groupby("order_id")
    .agg(
        total_items=("order_item_id", "count"),
        total_price=("price", "sum"),
        total_freight=("freight_value", "sum"),
        avg_item_price=("price", "mean")
    )
    .reset_index()
)

# 5. Join everything
orders_enriched = (
    orders_ok
    .merge(items_agg, on="order_id", how="left")
    .merge(reviews[["order_id", "review_score"]], on="order_id", how="left")
)


In [3]:
for name, df in {
    "customers": customers,
    "orders": orders,
    "items": items,
    "reviews": reviews,
    "payments": payments
}.items():
    print(f"{name}: {df.shape}")


customers: (99441, 5)
orders: (99441, 8)
items: (112650, 7)
reviews: (99224, 7)
payments: (103886, 5)


In [4]:
customers[["customer_id", "customer_unique_id"]].nunique()

customer_id           99441
customer_unique_id    96096
dtype: int64

In [5]:
orders_ok = orders[
    (orders["order_status"] == "delivered") &
    (orders["order_delivered_customer_date"].notna())
].copy()

orders_ok.shape

(96470, 8)

In [6]:
orders_ok = orders_ok.merge(
    customers[["customer_id", "customer_unique_id"]],
    on="customer_id",
    how="left"
)

orders_ok["customer_unique_id"].isna().sum()

np.int64(0)

In [7]:
items_agg = (
    items
    .groupby("order_id")
    .agg(
        total_items=("order_item_id", "count"),
        total_price=("price", "sum"),
        total_freight=("freight_value", "sum"),
        avg_item_price=("price", "mean")
    )
    .reset_index()
)

In [8]:
reviews_agg = (
    reviews
    .groupby("order_id")
    .agg(
        review_score=("review_score", "mean")
    )
    .reset_index()
)

In [9]:
orders_enriched = (
    orders_ok
    .merge(items_agg, on="order_id", how="left")
    .merge(reviews_agg, on="order_id", how="left")
)

In [10]:
orders_enriched["order_purchase_timestamp"] = pd.to_datetime(
    orders_enriched["order_purchase_timestamp"],
    errors="raise"
)

In [11]:
assert orders_enriched["order_id"].is_unique
assert orders_enriched["customer_unique_id"].notna().all()
assert (orders_enriched["total_price"] >= 0).all()

In [12]:
orders_enriched["total_price"].describe()
orders_enriched["review_score"].value_counts(dropna=False)

review_score
5.000000    56697
4.000000    18868
1.000000     9312
3.000000     7915
2.000000     2916
NaN           646
4.500000       53
2.500000       30
3.500000       23
1.500000        8
3.333333        1
4.333333        1
Name: count, dtype: int64

In [13]:
reference_date = orders_enriched["order_purchase_timestamp"].max()
reference_date

Timestamp('2018-08-29 15:00:37')

# Customer Segmentation

In [14]:
customer_features = (
    orders_enriched
    .groupby("customer_unique_id")
    .agg(
        n_orders=("order_id", "count"),
        total_spend=("total_price", "sum"),
        avg_order_value=("total_price", "mean"),
        avg_items_per_order=("total_items", "mean"),
        avg_item_price=("avg_item_price", "mean"),

        first_purchase=("order_purchase_timestamp", "min"),
        last_purchase=("order_purchase_timestamp", "max"),
    )
    .reset_index()
)

In [15]:
customer_features["recency_days"] = (
    reference_date - customer_features["last_purchase"]
).dt.days

customer_features["customer_lifetime_days"] = (
    customer_features["last_purchase"] - customer_features["first_purchase"]
).dt.days

In [16]:
customer_features[[
    "n_orders",
    "total_spend",
    "recency_days"
]].describe(percentiles=[0.25, 0.5, 0.75, 0.9])

,n_orders,total_spend,recency_days
count,93350.000000,93350.000000,93350.000000
mean,1.033423,141.620235,236.950070
std,0.209106,215.702028,152.589932
min,1.000000,0.850000,0.000000
25%,1.000000,47.650000,113.000000
50%,1.000000,89.700000,218.000000
75%,1.000000,154.697500,345.000000
90%,1.000000,279.990000,465.000000
max,15.000000,13440.000000,713.000000


In [17]:
q_orders = customer_features["n_orders"].quantile([0.5, 0.75])
q_spend  = customer_features["total_spend"].quantile([0.5, 0.75])
q_rec    = customer_features["recency_days"].quantile([0.25, 0.5])

In [18]:
def customer_segment(row):
    if row.n_orders >= 3:
        return "Repeat customers"
    if row.n_orders == 2:
        return "Returning customers"
    if row.n_orders == 1 and row.total_spend >= customer_features["total_spend"].median():
        return "High value one-time"
    return "Low value one-time"

In [19]:
customer_features["customer_segment"] = customer_features.apply(
    customer_segment, axis=1
)

In [20]:
customer_features["customer_segment"].value_counts()

customer_segment
Low value one-time     46198
High value one-time    44351
Returning customers     2573
Repeat customers         228
Name: count, dtype: int64

In [21]:
customer_features.groupby("customer_segment")[[
    "n_orders",
    "total_spend",
    "recency_days"
]].mean()

,n_orders,total_spend,recency_days
customer_segment,,,
High value one-time,1.000000,232.599135,236.653018
Low value one-time,1.000000,47.098055,238.305706
Repeat customers,3.399123,426.025658,201.280702
Returning customers,2.000000,245.345861,220.890789


# Product Segmentation

In [22]:
product_features = (
    items
    .groupby("product_id")
    .agg(
        n_orders=("order_id", "nunique"),
        total_units=("order_item_id", "count"),
        total_revenue=("price", "sum"),
        avg_price=("price", "mean"),
    )
    .reset_index()
)

In [23]:
product_features.describe(percentiles=[0.5, 0.75, 0.9])

,n_orders,total_units,total_revenue,avg_price
count,32951.000000,32951.000000,32951.000000,32951.000000
mean,3.108403,3.418713,412.480462,145.302464
std,9.456937,10.619709,1371.945598,246.895756
min,1.000000,1.000000,2.200000,0.850000
50%,1.000000,1.000000,136.750000,79.000000
75%,2.000000,3.000000,329.000000,154.900000
90%,6.000000,6.000000,801.600000,295.225000
max,467.000000,527.000000,63885.000000,6735.000000


In [24]:
items_with_customers = (
    items
    .merge(
        orders_enriched[["order_id", "customer_unique_id"]],
        on="order_id",
        how="left"
    )
    .merge(
        customer_features[["customer_unique_id", "customer_segment"]],
        on="customer_unique_id",
        how="left"
    )
)

In [25]:
product_segment_matrix = (
    items_with_customers
    .groupby(["product_id", "customer_segment"])
    .size()
    .unstack(fill_value=0)
)

In [26]:
def product_segment(row):
    if row["Repeat customers"] > row.sum() * 0.4:
        return "Loyalty products"
    if row["High value one-time"] > row.sum() * 0.5:
        return "Premium impulse"
    if row["Low value one-time"] > row.sum() * 0.5:
        return "Entry-level"
    return "Mixed audience"

In [27]:
product_segment_matrix

customer_segment,High value one-time,Low value one-time,Repeat customers,Returning customers
product_id,,,,
00066f42aeeb9f3007548bb9d3f33c38,1,0,0,0
00088930e925c41fd95ebfe695fd2655,1,0,0,0
0009406fd7479715e4bef61dd91f2462,1,0,0,0
000b8f95fcb9e0096488278317764d19,0,2,0,0
000d9be29b5207b54e86aa1b1ac54872,1,0,0,0
...,...,...,...,...
fff6177642830a9a94a0f2cba5e476d1,2,0,0,0
fff81cc3158d2725c0655ab9ba0f712c,1,0,0,0
fff9553ac224cec9d15d49f5a263411f,0,1,0,0


# Normalize product preferences

In [28]:
product_segment_share = product_segment_matrix.div(
    product_segment_matrix.sum(axis=1),
    axis=0
)

In [29]:
product_segment_share.head()
product_segment_share.describe()

customer_segment,High value one-time,Low value one-time,Repeat customers,Returning customers
count,32214.000000,32214.000000,32214.000000,32214.000000
mean,0.483372,0.446048,0.011110,0.059470
std,0.480644,0.477458,0.093937,0.206153
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.369396,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [30]:
def product_segment(row):
    if row["Repeat customers"] >= 0.4:
        return "Loyalty product"
    if row["Returning customers"] >= 0.4:
        return "Retention product"
    if row["High value one-time"] >= 0.5:
        return "Premium impulse"
    if row["Low value one-time"] >= 0.5:
        return "Entry-level"
    return "Mixed audience"

In [31]:
product_segments = (
    product_segment_share
    .apply(product_segment, axis=1)
    .rename("product_segment")
    .reset_index()
)

In [32]:
product_features = product_features.merge(
    product_segments,
    on="product_id",
    how="left"
)

In [33]:
product_features["product_segment"].value_counts()

product_segment
Premium impulse      15622
Entry-level          14261
Retention product     1883
Loyalty product        357
Mixed audience          91
Name: count, dtype: int64

In [34]:
recommendation_map = {
    "Low value one-time": ["Entry-level"],
    "High value one-time": ["Premium impulse"],
    "Returning customers": ["Retention product", "Mixed audience"],
    "Repeat customers": ["Loyalty product", "Mixed audience"],
}

In [35]:
def recommend_products(customer_id, top_n=10):
    segment = customer_features.loc[
        customer_features.customer_unique_id == customer_id,
        "customer_segment"
    ].iloc[0]

    target_segments = recommendation_map[segment]

    return (
        product_features[
            product_features.product_segment.isin(target_segments)
        ]
        .sort_values("total_revenue", ascending=False)
        .head(top_n)
    )

In [36]:
customer_products = (
    items_with_customers
    .groupby("customer_unique_id")["product_id"]
    .apply(set)
)

In [37]:
def recommend_products(customer_id, top_n=10):
    # 1. Identify customer segment
    segment = customer_features.loc[
        customer_features.customer_unique_id == customer_id,
        "customer_segment"
    ].iloc[0]

    # 2. Which product segments we allow for this customer
    target_segments = recommendation_map[segment]

    # 3. Build candidate set
    candidates = product_features[
        product_features.product_segment.isin(target_segments)
    ].copy()

    # 4. Attach affinity score (segment-aware!)
    affinity = (
        product_segment_share[[segment]]
        .rename(columns={segment: "affinity"})
        .reset_index()
    )

    candidates = candidates.merge(
        affinity,
        on="product_id",
        how="left"
    )

    # Missing affinity = 0
    candidates["affinity"] = candidates["affinity"].fillna(0)

    # 5. Rank: affinity first, revenue second
    return (
        candidates
        .sort_values(
            ["affinity", "total_revenue"],
            ascending=False
        )
        .head(top_n)
    )

In [38]:
customer_features["customer_segment"].sample(5)

17308    High value one-time
41000    High value one-time
83455    High value one-time
88730     Low value one-time
45054     Low value one-time
Name: customer_segment, dtype: object

In [39]:
customer_features[
    ["customer_unique_id", "customer_segment"]
].sample(10)

customer_id_1 = customer_features.loc[
    customer_features.customer_segment == "Low value one-time",
    "customer_unique_id"
].iloc[0]

customer_id_2 = customer_features.loc[
    customer_features.customer_segment == "Repeat customers",
    "customer_unique_id"
].iloc[0]

recommend_products(customer_id_1)
recommend_products(customer_id_2)

,product_id,n_orders,total_units,total_revenue,avg_price,product_segment,affinity
13,060965aa6dfa817b80abda0c9413cc77,2,2,1669.12,834.56,Loyalty product,1.0
370,d35d1fa56f3c5a2777f5542a84a085dd,1,1,1013.40,1013.40,Loyalty product,1.0
15,07755a0056db849613241c6d91eaf45f,1,1,849.00,849.00,Loyalty product,1.0
287,a3f487d2926f4af42e86454a8ec474c1,1,3,567.00,189.00,Loyalty product,1.0
184,65f037a133d9ffc71164a5d69dee187e,1,3,518.70,172.90,Loyalty product,1.0
194,6e2a68aa94eb2a3b5b3499bf2314d544,1,2,503.84,251.92,Loyalty product,1.0
298,ad1280b6c5b1af7cfd97372249be66c9,1,1,469.90,469.90,Loyalty product,1.0
24,0b00b16606aeb46da3543bbabff4d6b3,1,2,465.98,232.99,Loyalty product,1.0
111,3ed43e8f95f9801cd8099b6f73e07335,1,1,429.90,429.90,Loyalty product,1.0
166,596dd8bfc1289b4336ef857ffb960d47,1,1,429.90,429.90,Loyalty product,1.0


In [40]:
def sample_customers_by_segment(n=3):
    return (
        customer_features
        .groupby("customer_segment")
        .sample(n, random_state=42)
        [["customer_unique_id", "customer_segment"]]
    )

In [41]:
def recommend_products(customer_id, top_n=10):
    segment = customer_features.loc[
        customer_features.customer_unique_id == customer_id,
        "customer_segment"
    ].iloc[0]

    target_segments = recommendation_map[segment]

    return (
        product_features[
            (product_features.product_segment.isin(target_segments)) &
            (product_features.n_orders >= MIN_ORDERS)
        ]
        .sort_values("total_revenue", ascending=False)
        .head(top_n)
    )

In [42]:
customer_product_history = (
    items_with_customers
    .groupby(["customer_unique_id", "product_id"])
    .agg(
        n_purchases=("order_id", "count"),
        total_spent=("price", "sum")
    )
    .reset_index()
)

In [43]:
segment_product_affinity = (
    items_with_customers
    .groupby(["customer_segment", "product_id"])
    .size()
    .rename("segment_purchases")
    .reset_index()
)

In [44]:
segment_totals = (
    segment_product_affinity
    .groupby("customer_segment")["segment_purchases"]
    .sum()
    .rename("segment_total")
    .reset_index()
)

segment_product_affinity = segment_product_affinity.merge(
    segment_totals,
    on="customer_segment"
)

segment_product_affinity["affinity"] = (
    segment_product_affinity["segment_purchases"]
    / segment_product_affinity["segment_total"]
)

In [45]:
def recommend_products(customer_id, top_n=10):
    segment = customer_features.loc[
        customer_features.customer_unique_id == customer_id,
        "customer_segment"
    ].iloc[0]

    allowed_product_segments = recommendation_map[segment]

    candidates = (
        product_features[
            product_features.product_segment.isin(allowed_product_segments)
        ]
        .merge(
            segment_product_affinity[
                segment_product_affinity.customer_segment == segment
            ][["product_id", "affinity"]],
            on="product_id",
            how="left"
        )
        .fillna({"affinity": 0})
    )

    return (
        candidates
        .sort_values(
            ["affinity", "total_revenue"],
            ascending=False
        )
        .head(top_n)
    )

In [46]:
recommend_products(customer_id_1)
recommend_products(customer_id_2)

,product_id,n_orders,total_units,total_revenue,avg_price,product_segment,affinity
431,f3720bc68555b1bff49b9ffd41b017ac,7,20,3627.50,181.375000,Loyalty product,0.011976
83,2fb9e46750ac55362f7b642f12b5835b,2,8,120.00,15.000000,Loyalty product,0.007984
292,a8d2c5e8f29550a539f377d977f10a52,5,7,638.00,91.142857,Loyalty product,0.005988
201,70906e04da1eebf3d1b8791bd09ffe85,12,17,1193.39,70.199412,Mixed audience,0.004990
87,325e75d20ca67d859f707129be35878e,2,5,224.95,44.990000,Loyalty product,0.004990
342,c2b534c5a4a6cbfc41aeaf362fb0c060,1,5,102.50,20.500000,Loyalty product,0.004990
289,a5b15c8b0abb9a1e7eb10546441925d0,3,5,60.00,12.000000,Loyalty product,0.004990
318,b655ebf10fa7727c97d82cffcfe96ab9,1,5,47.50,9.500000,Loyalty product,0.004990
1,00faa46f36261af8bbf3a4d37fa4841b,10,10,2800.00,280.000000,Loyalty product,0.003992
59,202bd859659a841de892b00c341300ff,11,13,455.00,35.000000,Mixed audience,0.003992


In [47]:
def recommend_products(customer_id, top_n=10):
    segment = customer_features.loc[
        customer_features.customer_unique_id == customer_id,
        "customer_segment"
    ].iloc[0]

    target_segments = recommendation_map[segment]

    # products already bought by this customer
    bought_products = customer_product_history.loc[
        customer_product_history.customer_unique_id == customer_id,
        "product_id"
    ]

    candidates = (
        product_features[
            product_features.product_segment.isin(target_segments)
        ]
        .merge(
            segment_product_affinity[
                segment_product_affinity.customer_segment == segment
            ][["product_id", "affinity"]],
            on="product_id",
            how="left"
        )
        .fillna({"affinity": 0})
    )

    candidates = candidates[
        ~candidates.product_id.isin(bought_products)
    ]

    return (
        candidates
        .sort_values(
            ["affinity", "total_revenue"],
            ascending=False
        )
        .head(top_n)
    )

In [48]:
# 1. Compare two customers from different segments
recommend_products(customer_features.customer_unique_id.iloc[0])
recommend_products(customer_features.customer_unique_id.iloc[-1])

,product_id,n_orders,total_units,total_revenue,avg_price,product_segment,affinity
9525,aca2eb7d00ea1a7b8ebd4e68314663af,431,527,37608.90,71.364137,Entry-level,0.006669
1199,154e7e31ebfa092203795c972e5804a6,269,281,6325.19,22.509573,Entry-level,0.004480
3147,389d119b48cf3043d311335e499d9c6b,311,392,21440.59,54.695383,Entry-level,0.004075
3015,368c6c730842d78016ad823897a372db,291,388,21056.80,54.270103,Entry-level,0.003933
6932,7c1bd920dbdf22470b68bde975dd3ccf,225,231,13866.69,60.028961,Entry-level,0.003689
9317,a92930c327948861c015c919a0bcb4a8,158,160,12475.00,77.968750,Entry-level,0.002939
604,0aabfb375647d9738ad0f7b4ea3653b1,138,142,3416.70,24.061268,Entry-level,0.002757
3544,3fbc0ef745950c7932d5f2a446189725,144,150,9862.50,65.750000,Entry-level,0.002635
12467,e0cf79767c5b016251fe139915c59a26,131,137,4096.30,29.900000,Entry-level,0.002635
397,06edb72f1e0c64b14c5b79353f7abea3,130,143,5831.77,40.781608,Entry-level,0.002534


In [49]:
items_with_customers = (
    items_with_customers
    .merge(
        orders_enriched[["order_id", "order_purchase_timestamp"]],
        on="order_id",
        how="left"
    )
)

In [50]:
items_with_customers["order_purchase_timestamp"].isna().sum()

np.int64(2461)

In [51]:
items_with_customers["days_ago"] = (
    reference_date - items_with_customers["order_purchase_timestamp"]
).dt.days

In [52]:
import numpy as np

LAMBDA = 0.01   # decay speed (safe default)

items_with_customers["time_weight"] = np.exp(
    -LAMBDA * items_with_customers["days_ago"]
)

In [53]:
items_with_customers[["order_purchase_timestamp", "days_ago", "time_weight"]].head()

,order_purchase_timestamp,days_ago,time_weight
0,2017-09-13 08:59:02,350.0,0.030197
1,2017-04-26 10:53:06,490.0,0.007447
2,2018-01-14 14:33:31,227.0,0.103312
3,2018-08-08 10:00:35,21.0,0.810584
4,2017-02-04 13:57:51,571.0,0.003313


In [54]:
def recommend_products_static(customer_id, top_n=10):
    segment = customer_features.loc[
        customer_features.customer_unique_id == customer_id,
        "customer_segment"
    ].iloc[0]

    target_segments = recommendation_map[segment]

    return (
        product_features[
            product_features.product_segment.isin(target_segments)
        ]
        .sort_values("total_revenue", ascending=False)
        .head(top_n)
        .assign(method="static")
    )

In [55]:
product_affinity_time = (
    items_with_customers
    .groupby(["product_id", "customer_segment"])
    .agg(
        affinity=("time_weight", "sum")
    )
    .reset_index()
)

In [56]:
product_affinity_time["affinity"] = (
    product_affinity_time
    .groupby("customer_segment")["affinity"]
    .transform(lambda x: x / x.sum())
)

In [57]:
product_affinity_time = product_affinity_time.merge(
    product_features,
    on="product_id",
    how="left"
)

In [58]:
def recommend_products_timeaware(customer_id, top_n=10):
    segment = customer_features.loc[
        customer_features.customer_unique_id == customer_id,
        "customer_segment"
    ].iloc[0]

    target_segments = recommendation_map[segment]

    return (
        product_affinity_time[
            (product_affinity_time.customer_segment == segment) &
            (product_affinity_time.product_segment.isin(target_segments))
        ]
        .sort_values("affinity", ascending=False)
        .head(top_n)
        .assign(method="time_aware")
    )

In [59]:
customer_id = customer_features.customer_unique_id.sample(1).iloc[0]

In [60]:
static_rec = recommend_products_static(customer_id)
time_rec   = recommend_products_timeaware(customer_id)

comparison = pd.concat([
    static_rec[["product_id", "total_revenue", "product_segment", "method"]],
    time_rec[["product_id", "affinity", "product_segment", "method"]],
])

comparison

,product_id,total_revenue,product_segment,method,affinity
8613,422879e10f46682990de24d770e7f83d,26577.22,Mixed audience,static,NaN
10840,53759a2ecddad2bb87a079a1f1519f73,20387.20,Mixed audience,static,NaN
8227,3f14d740544f37ece8a9e7bc8349797e,7731.03,Mixed audience,static,NaN
28327,dbb67791e405873b259e4656bf971246,6975.37,Mixed audience,static,NaN
21316,a6492cc69376c469ab6f61d8f44de961,4799.00,Retention product,static,NaN
7032,362b773250263786dd58670d2df42c3b,3782.70,Mixed audience,static,NaN
25553,c6aab69ca5c62eb16eafaddf36c38ccd,3651.80,Retention product,static,NaN
28325,dbb4ce89c8ed5fb6fd901e2e51093179,3623.95,Mixed audience,static,NaN
19262,962a6951154f98f2c8e9a5b8b2bcf4a9,3459.80,Mixed audience,static,NaN
23071,b3cd6ea8ef75cbcb2856861b31eb9bc0,2853.97,Retention product,static,NaN


# Liczba zamówień na klienta

In [61]:
customer_features["n_orders"].value_counts().head()

n_orders
1    90549
2     2573
3      181
4       28
5        9
Name: count, dtype: int64

# Rozkład wartości zamówień

In [62]:
orders_enriched["total_price"].describe()

count    96470.000000
mean       137.040001
std        209.052608
min          0.850000
25%         45.900000
50%         86.500000
75%        149.900000
max      13440.000000
Name: total_price, dtype: float64

# Top produkty wg przychodu

In [63]:
product_features.sort_values("total_revenue", ascending=False).head(10)

,product_id,n_orders,total_units,total_revenue,avg_price,product_segment
24086,bb50f2e236e5eea0100680137654686c,187,195,63885.00,327.615385,Premium impulse
14068,6cdd53843498f92890544667809f1595,151,156,54730.20,350.834615,Premium impulse
27613,d6160fb7873f184099d9bc95e30376af,35,35,48899.34,1397.124000,Premium impulse
27039,d1c427060a0f73f6b889a5c7c61f2ac4,323,343,47214.51,137.651633,Premium impulse
19742,99a4788cb24856965c36a24e339b6058,467,488,43025.56,88.167131,Premium impulse
8051,3dd2a17168ec895c781a9191c1e95ad7,255,274,41082.60,149.936496,Premium impulse
4996,25c38557cf793876c5abdd5931f922db,38,38,38907.32,1023.876842,Premium impulse
12351,5f504b3a1c75b73d6151be81eb05bdc9,63,63,37733.90,598.950794,Premium impulse
10867,53b36df67ebb7c41585e8d54d6772e08,306,323,37683.42,116.666935,Premium impulse
22112,aca2eb7d00ea1a7b8ebd4e68314663af,431,527,37608.90,71.364137,Entry-level


# Udział segmentów klientów

In [64]:
customer_features["customer_segment"].value_counts(normalize=True)

customer_segment
Low value one-time     0.494890
High value one-time    0.475104
Returning customers    0.027563
Repeat customers       0.002442
Name: proportion, dtype: float64

# Udział segmentów produktów

In [65]:
product_features["product_segment"].value_counts()

product_segment
Premium impulse      15622
Entry-level          14261
Retention product     1883
Loyalty product        357
Mixed audience          91
Name: count, dtype: int64

```
Raw E-commerce Data
(customers, orders, items, reviews)
          |
          v
Data Cleaning & Filtering
(only delivered orders)
          |
          v
Feature Engineering
- Customer features
- Product features
          |
          v
Customer Segmentation
(rule-based, interpretable)
          |
          v
Product Segmentation
(based on customer mix)
          |
          v
Recommendation Logic
(customer segment → product segment)
          |
          v
Ranked Product Recommendations
```

```
[1] Raw Data
    ├─ Customers
    ├─ Orders
    ├─ Order Items
    └─ Reviews

[2] Data Preparation
    ├─ Filter delivered orders
    ├─ Join tables
    └─ Aggregate to order level

[3] Feature Engineering
    ├─ Customer features
    │   ├─ n_orders
    │   ├─ total_spend
    │   ├─ recency_days
    │   └─ lifetime
    └─ Product features
        ├─ n_orders
        ├─ total_units
        ├─ total_revenue
        └─ avg_price

[4] Segmentation
    ├─ Customer segmentation
    └─ Product segmentation

[5] Recommendation Engine
    ├─ Match customer → product segments
    ├─ Rank products
    └─ Output recommendations

```

In [67]:
# Zapisujemy gotowe wyniki do folderu data
customer_features.to_csv("data/customer_segments.csv", index=False)
product_features.to_csv("data/product_segments.csv", index=False)
segment_product_affinity.to_csv("data/segment_affinity.csv", index=False)

In [71]:
import pandas as pd
import os

print("🔄 Rozpoczynam generowanie pełnej historii zamówień...")

# 1. WCZYTUJEMY TABELE OD ZERA (Żeby mieć 100% pewności)
# Używamy surowych plików, żeby niczego nie brakowało
items = pd.read_csv("data/olist_order_items_dataset.csv")
orders = pd.read_csv("data/olist_orders_dataset.csv")
customers = pd.read_csv("data/olist_customers_dataset.csv")
products = pd.read_csv("data/olist_products_dataset.csv")

# 2. ŁĄCZYMY WSZYSTKO W JEDNĄ CAŁOŚĆ
# Krok A: Połącz przedmioty z zamówieniami (żeby mieć daty)
history = items.merge(orders[['order_id', 'customer_id', 'order_purchase_timestamp']], on='order_id')

# Krok B: Połącz z klientami (żeby mieć customer_unique_id)
history = history.merge(customers[['customer_id', 'customer_unique_id']], on='customer_id')

# Krok C: Połącz z produktami (żeby mieć nazwy kategorii - to tutaj wcześniej brakowało danych)
history = history.merge(products[['product_id', 'product_category_name']], on='product_id', how='left')

# 3. TŁUMACZENIE NA ANGIELSKI (OPCJONALNE ALE WARTO)
# Sprawdzamy, czy masz plik z tłumaczeniami (widziałem go na screenie)
trans_file = "data/product_category_name_translation.csv"
if os.path.exists(trans_file):
    translations = pd.read_csv(trans_file)
    # Łączymy tłumaczenia
    history = history.merge(translations, on='product_category_name', how='left')
    # Jeśli jest angielska nazwa, bierzemy ją. Jak nie - zostaje portugalska. Jak brak obu - "Unknown".
    history['category_display'] = history['product_category_name_english'].fillna(history['product_category_name']).fillna("Unknown")
else:
    history['category_display'] = history['product_category_name'].fillna("Unknown")

# 4. CZYSZCZENIE I EKSPORT
# Formatujemy nazwy (zamiana podkreśleń na spacje, duże litery)
history['product_category_name'] = history['category_display'].str.replace('_', ' ').str.title()

final_export = history[[
    'customer_unique_id',
    'order_purchase_timestamp',
    'product_category_name',
    'price',
    'product_id'
]].sort_values('order_purchase_timestamp', ascending=False)

# Zapisujemy
final_export.to_csv("data/customer_order_history.csv", index=False)

print(f"✅ Sukces! Zapisano {len(final_export)} wierszy historii.")
print("Przykładowe kategorie:", final_export['product_category_name'].unique()[:5])

🔄 Rozpoczynam generowanie pełnej historii zamówień...
✅ Sukces! Zapisano 112650 wierszy historii.
Przykładowe kategorie: ['Kitchen Dining Laundry Garden Furniture' 'Computers Accessories'
 'Party Supplies' 'Health Beauty' 'Toys']


In [72]:
import pandas as pd
import numpy as np

# 1. Przygotowanie bazy produktów z nazwami (żeby nie było tylko ID)
# Łączymy produkty z ich kategoriami
if 'products' in locals():
    prod_data = products[['product_id', 'product_category_name']].copy()

    # Opcjonalne tłumaczenie na angielski
    if os.path.exists("data/product_category_name_translation.csv"):
        trans = pd.read_csv("data/product_category_name_translation.csv")
        prod_data = prod_data.merge(trans, on='product_category_name', how='left')
        prod_data['display_name'] = prod_data['product_category_name_english'].fillna(prod_data['product_category_name'])
    else:
        prod_data['display_name'] = prod_data['product_category_name']

    prod_data['display_name'] = prod_data['display_name'].str.replace('_', ' ').str.title()
    prod_data = prod_data.fillna("General Product")
else:
    # Jeśli nie ma tabeli products, tworzymy sztuczne nazwy
    prod_data = pd.DataFrame(items['product_id'].unique(), columns=['product_id'])
    prod_data['display_name'] = "Product " + prod_data['product_id'].str[-5:]

# 2. Budujemy nową tabelę Affinity na poziomie PRODUKTU, a nie kategorii
# Łączymy items (produkty) z klientami i ich segmentami
reco_base = items.merge(orders[['order_id', 'customer_id']], on='order_id')
reco_base = reco_base.merge(customers[['customer_id', 'customer_unique_id']], on='customer_id')
reco_base = reco_base.merge(customer_features[['customer_unique_id', 'customer_segment']], on='customer_unique_id')

# Liczymy popularność każdego PRODUKTU w każdym segmencie
product_affinity = reco_base.groupby(['customer_segment', 'product_id']).size().reset_index(name='sales_count')

# Dodajemy nazwy (display_name)
product_affinity = product_affinity.merge(prod_data[['product_id', 'display_name']], on='product_id', how='left')

# Obliczamy Match Score (0-100) wewnątrz każdego segmentu
max_sales = product_affinity.groupby('customer_segment')['sales_count'].transform('max')
product_affinity['match_score'] = (product_affinity['sales_count'] / max_sales * 98).round(1)

# Sortujemy, żeby mieć pewność że Top produkty są na górze
product_affinity = product_affinity.sort_values(['customer_segment', 'sales_count'], ascending=[True, False])

# Zapisujemy nową tabelę
product_affinity.to_csv("data/product_affinity_v2.csv", index=False)
print("✅ Wygenerowano plik product_affinity_v2.csv z konkretnymi produktami!")

✅ Wygenerowano plik product_affinity_v2.csv z konkretnymi produktami!


In [73]:
import pandas as pd
import os

# Ręczna lista tłumaczeń (portugalski -> angielski)
# To są najpopularniejsze kategorie w Olist
data = {
    'product_category_name': [
        'beleza_saude', 'informatica_acessorios', 'automotivo', 'cama_mesa_banho',
        'moveis_decoracao', 'esporte_lazer', 'perfumaria', 'utilidades_domesticas',
        'telefonia', 'relogios_presentes', 'alimentos_bebidas', 'bebes',
        'papelaria', 'tablets_impressao_imagem', 'brinquedos', 'telefonia_fixa',
        'ferramentas_jardim', 'fashion_bolsas_e_acessorios', 'eletroportateis',
        'consoles_games', 'audio', 'fashion_calcados', 'cool_stuff',
        'malas_acessorios', 'climatizacao', 'construcao_ferramentas_construcao',
        'moveis_cozinha_area_de_servico_jantar_e_jardim', 'moveis_escritorio',
        'eletronicos', 'pet_shop', 'pcs'
    ],
    'product_category_name_english': [
        'health_beauty', 'computers_accessories', 'auto', 'bed_bath_table',
        'furniture_decor', 'sports_leisure', 'perfumery', 'housewares',
        'telephony', 'watches_gifts', 'food_drink', 'baby',
        'stationery', 'tablets_printing_image', 'toys', 'fixed_telephony',
        'garden_tools', 'fashion_bags_accessories', 'small_appliances',
        'consoles_games', 'audio', 'fashion_shoes', 'cool_stuff',
        'luggage_accessories', 'air_conditioning', 'construction_tools_construction',
        'kitchen_dining_laundry_garden_furniture', 'office_furniture',
        'electronics', 'pet_shop', 'computers'
    ]
}

# Tworzymy DataFrame
df_trans = pd.DataFrame(data)

# Zapisujemy do pliku
path = "data/product_category_name_translation.csv"
df_trans.to_csv(path, index=False)

print(f"✅ Słownik tłumaczeń został utworzony/nadpisany w: {path}")
print(f"Liczba kategorii w słowniku: {len(df_trans)}")

✅ Słownik tłumaczeń został utworzony/nadpisany w: data/product_category_name_translation.csv
Liczba kategorii w słowniku: 31
